# Model Training
> To facilitate a more automated training procedure, the model training is moved to a standalone python script.  
This keeps Keras much happier in terms of required restarts and memory usage.

In [1]:
%load_ext autoreload
%autoreload 2
from importlib import reload

import numpy as np
import time # !
import json
from matplotlib import pyplot as plt
import time

from keras.utils import to_categorical

import glove_helper
from loadutils import conll2003Data, saveProcessedData, retrieve_model
from common import vocabulary, utils


Using TensorFlow backend.


In [2]:

start_time = time.time()

"""
Pick which language to train on
"""
LANGUAGE = "es"
# LANGUAGE = "ca"
# LANGUAGE = "gd"

"""
Pick how much of the training data to use
"""
# TRAIN_AMOUNT = "100"
# TRAIN_AMOUNT = "50"
TRAIN_AMOUNT = "10"

DIRECTORY = "../data/pos_tagging/"+LANGUAGE+"/"

#training file depends on low-resource or not; 100%, 50%, or 10% of training data used
TRAIN_FILE = DIRECTORY+"train_"+TRAIN_AMOUNT+".txt"

# dev, test, and vectors stay the same
DEV_FILE = DIRECTORY+"dev.txt"
TEST_FILE = DIRECTORY+"test.txt"
# VECTORS = "data/"+LANGUAGE+"/wiki."+LANGUAGE+".zip"
VECTORS = "data/"+LANGUAGE+"/cc."+LANGUAGE+".300.zip"

# out files for IPC
HYPER_PARAM_FILE = "hyper_params.json"

VOCAB_SIZE = 200000

## PRINT OUT HYPERPARAMETERS FOR REFERENCE
print("LANGUAGE:\t", LANGUAGE)
print("TRAIN_AMOUNT:\t", TRAIN_AMOUNT)

LANGUAGE:	 es
TRAIN_AMOUNT:	 10


## Local helper utils

In [3]:
# local untils

# timeit decorator
def timeit(method):
    def timed(*args, **kw):
        ts = time.time()
        result = method(*args, **kw)
        te = time.time()
        if 'log_time' in kw:
            name = kw.get('log_name', method.__name__.upper())
            kw['log_time'][name] = int((te - ts) * 1000)
        else:
            print ('%r  %2.2f ms' % \
                  (method.__name__, (te - ts) * 1000))
        return result
    return timed

In [4]:
def construct_embedding_matrix(embed_dim, vocab_size):
    """
    construct embedding matrix from GloVe 6Bn word data
    
    reuse glove_helper code from w266 
    
    Returns: an embedding matrix directly plugged into keras.layers.Embedding(weights=[embedding_matrix])
    """
    reload(glove_helper)
    hands = glove_helper.Hands(vector_zip=VECTORS, ndim=embed_dim)
    embedding_matrix = np.zeros((vocab_size, embed_dim))
    
    for i in range(vocabData.vocab.size):
        word = vocabData.vocab.ids_to_words([i])[0]
        try:
            embedding_vector = hands.get_vector(word)
        except:
            embedding_vector = hands.get_vector("<unk>")
        embedding_matrix[i] = embedding_vector

    return embedding_matrix

In [5]:
def plot_history( history):
    # summarize history for accuracy
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    
    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

## Load the Data

In [6]:
# UPDATES!
global_max_features = 200000
windowLength = 11
#testNumSents = 20000

# Use training set to build vocab here
vocabData = conll2003Data(TRAIN_FILE)
vocabData.buildVocab( vocabSize=global_max_features, verbose=False)

----------------------------------------------------
reading file from path ../data/pos_tagging/es/train_10.txt
'readFile'  155.35 ms
----------------------------------------------------
building vocabulary from TRAINING data...
vocabulary for words, posTags, nerTags built and stored in object
vocab size = 200000
10 sampled words from vocabulary
 ['c', 'deudas', 'accidentes', 'parecerían', 'repartido', 'distribuye', 'circulando', 'albal', 'brotons', 'films'] 

number of unique pos Tags in training = 67
all posTags used
 ['Mood=Cnd', 'Tense=Past', 'NumType=Frac', '<unk>', 'Person=2', 'Mood=Ind', 'PunctType=Brck', 'Gender=Fem', 'Number=Plur', 'PrepCase=Pre', 'PronType=Art', 'PronType=Prs', 'Definite=Ind', 'Polarity=Neg', 'PunctType=Excl', 'Poss=Yes', 'Case=Dat', 'Person=1', 'Mood=Imp', 'VerbForm=Inf', 'PunctType=Colo', 'Number[psor]=Plur', 'Number[psor]=Sing', 'Mood=Sub', 'Polite=Form', 'Tense=Imp', 'Gender=Masc', 'Tense=Fut', '</s>', 'NumForm=Digit', 'PronType=Dem', 'PrepCase=Npr', 'Ver

In [7]:
# Format training data
trainX, trainX_pos, trainX_capitals, trainY  = vocabData.formatWindowedData( 
                                                  vocabData.train_sentences, 
                                                  windowLength=windowLength,
                                                  verbose=False)

# read in dev data
devSents = vocabData.readFile( DEV_FILE)
devX, devX_pos, devX_capitals, devY = vocabData.formatWindowedData( 
                                              devSents, 
                                              windowLength=windowLength,
                                              verbose=False)

# read in the test data
testSents = vocabData.readFile( TEST_FILE)
testX, testX_pos, testX_capitals, testY = vocabData.formatWindowedData( 
                                                testSents, 
                                                windowLength=windowLength,
                                                verbose=False)

----------------------------------------------------
formatting sentences into input windows...
'formatWindowedData'  586.24 ms
----------------------------------------------------
reading file from path ../data/pos_tagging/es/dev.txt
'readFile'  197.42 ms
----------------------------------------------------
formatting sentences into input windows...
'formatWindowedData'  697.88 ms
----------------------------------------------------
reading file from path ../data/pos_tagging/es/test.txt
'readFile'  323.73 ms
----------------------------------------------------
formatting sentences into input windows...
'formatWindowedData'  585.77 ms


In [8]:
# Load GloVe embedding matrix

# setting it to global instead of hyper_param dictionaries because embedding \
# dimensions need to be decided before the data is loaded for the decoder output
# global_embed_dim = 50
global_embed_dim = 300

embedding_matrix = construct_embedding_matrix( global_embed_dim, 
                                               global_max_features)

Loading vectors from data/es/cc.es.300.zip
Parsing file: data/es/cc.es.300.zip:cc.es.300.vec
Found 2,000,000 words.
Parsing vectors... Done! (W.shape = (2000003, 300))


In [9]:
# Get Y

# encoding 1-hot for ner targets
trainY_cat = to_categorical(trainY.astype('float32'))
devY_cat = to_categorical(devY.astype('float32'), num_classes=trainY_cat.shape[1])
testY_cat = to_categorical(testY.astype('float32'), num_classes=trainY_cat.shape[1])

trainY_cat = np.array(list(map( lambda i: np.array(i[3:], dtype=np.float), trainY_cat)), dtype=np.float)
devY_cat = np.array(list(map( lambda i: np.array(i[3:], dtype=np.float), devY_cat)), dtype=np.float)
testY_cat = np.array(list(map( lambda i: np.array(i[3:], dtype=np.float), testY_cat)), dtype=np.float)

In [10]:
# Get decoder Y -- 50 dim embedding of center word

train_decoderY = embedding_matrix[trainX[:,4]]
dev_decoderY = embedding_matrix[devX[:,4]]
test_decoderY = embedding_matrix[testX[:,4]]

In [11]:
# trainX => windows of size 11, center word is what we're predicting the label for
# trainY => tag/prediction for center word only!
# train_decoderY[0]

In [13]:
# our take on keras's to_categorical method, but if we DON'T want a one-hot vector
# instead, the values of the vectors are 1 for ALL of the features present, so one token's vector
# could have a 1 in more than one dimension here

def to_almost_categorical(y, num_classes):
    almost_categorical_array = np.zeros((y.shape[0], y.shape[1], num_classes))
#     print(almost_categorical_array.shape)
    for i in range(len(y[0])):
#         print("window:\t",window)
        window = y[i]
        window_index = i
#         print("window_index:\t", i)
#         print("window:\t", window)
        for j in range(len(window)):
            token = window[j]
            token_index = j
#             print("token_index:\t", token_index)
#             print("token:\t",token)
            for value in token:
#                 print("value:\t",value)
#                 print("token_index:\t",token_index)
                value = int(value)
                almost_categorical_array[i,token_index,value] += 1
#                 print(almost_categorical_array[i])
    print(almost_categorical_array)
    return almost_categorical_array
        

In [14]:
# Get X pos tags

# encoding 1-hot for pos tags
trainX_pos_cat = to_almost_categorical(trainX_pos, num_classes=vocabData.posTags.size)
devX_pos_cat = to_almost_categorical(devX_pos, num_classes=vocabData.posTags.size) 
testX_pos_cat = to_almost_categorical(testX_pos, num_classes=vocabData.posTags.size)

[[[1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 ...

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]
[[[1. 0. 0

In [15]:
# encoding 1-hot for pos tags
# trainX_pos_cat = to_categorical(trainX_pos.astype('float32'))
# devX_pos_cat = to_categorical(devX_pos.astype('float32'), num_classes=trainX_pos_cat.shape[2]) 
# testX_pos_cat = to_categorical(testX_pos.astype('float32'), num_classes=trainX_pos_cat.shape[2])

In [16]:
"""
trying to get pos tags crashes the kernel on my laptop, try on another device or on HPC
"""

trainX_pos_cat = np.array(list(map( lambda i: np.array(i[:,3:], dtype=np.float), trainX_pos_cat)), dtype=np.float)

In [17]:
devX_pos_cat = np.array(list(map( lambda i: np.array(i[:,3:], dtype=np.float), devX_pos_cat)), dtype=np.float)

In [18]:
testX_pos_cat = np.array(list(map( lambda i: np.array(i[:,3:], dtype=np.float), testX_pos_cat)), dtype=np.float)

In [24]:
# Get X capitalization 

# encoding 1-hot for capitalization info  ("allCaps", "upperInitial", "lowercase", "mixedCaps", "noinfo")
trainX_capitals_cat = to_categorical(trainX_capitals.astype('float32'))
devX_capitals_cat = to_categorical(devX_capitals.astype('float32'), num_classes=trainX_capitals_cat.shape[2]) 
testX_capitals_cat = to_categorical(testX_capitals.astype('float32'), num_classes=trainX_capitals_cat.shape[2])

In [25]:
trainX_capitals_cat = np.array(list(map( lambda i: np.array(i[:,3:], dtype=np.float), trainX_capitals_cat)), dtype=np.float)
devX_capitals_cat = np.array(list(map( lambda i: np.array(i[:,3:], dtype=np.float), devX_capitals_cat)), dtype=np.float)
testX_capitals_cat = np.array(list(map( lambda i: np.array(i[:,3:], dtype=np.float), testX_capitals_cat)), dtype=np.float)

## Set up model parameters

In [26]:
# define hyper parameters for model
# CAPSNET
hyper_param_caps = {
    
    'max_features' : global_max_features,  # 20000
    'maxlen' : trainX.shape[1],  # window size (11)
    'poslen' : trainX_pos_cat.shape[2],  # pos classes (216)
    # TODO: turn this into feature types, or only use that for capsule dimension?
    'capitallen' : trainX_capitals_cat.shape[2],  # capitalization classes (5)
    'ner_classes' : trainY_cat.shape[1],  # 18, POS coarse tags 
    'embed_dim' : global_embed_dim,  # word embedding size (300)
    'num_routing' : 3, 

    'use_glove' : True,
    'allow_glove_retrain' : False,
    'use_pos_tags' : True,
    'use_capitalization_info' : True,    
    
    'conv1_filters' : 256,
    'conv1_kernel_size' : 3,
    'conv1_strides' : 1,
    'conv1_padding' : 'valid',
    
    'use_2D_primarycaps' : False,
    'primarycaps_dim_capsule' : 8,
#     'primarycaps_dim_capsule' : 22, # number of feature types?
    'primarycaps_n_channels' : 32,
    'primarycaps_kernel_size' : 3,
    'primarycaps_strides' : 1,
    'primarycaps_padding' : 'valid',

    'ner_capsule_dim' : 16,
#     'ner_capsule_dim' : 18, # number of coarse POS tags?

    
    'num_dynamic_routing_passes' : 3,
    
    # decoder is still work in progress
    'use_decoder' : False,
    'decoder_feed_forward_1' : 100,
    'decoder_feed_forward_2' : 100, 
    'decoder_dropout' : 0.3,
    
    'save_dir' : './result',
    'batch_size' : 100,
    'debug' : 2,
    'epochs' : 5,
    'stopping_patience' : 2, 
    'dropout_p' : 0.25,
    'embed_dropout' : 0.25,
    'lam_recon' : 0.0005,
    
    'optimizer' : 'Adam', #or 'SGD'
    'loss_function' : 'custom_cosine', # mean_squared_error
}

In [27]:
# define hyper parameters for model
# CNN
hyper_param_cnn = {
    
    'max_features' : global_max_features,  # 20000
    'maxlen' : trainX.shape[1],  # window size (11)
    'poslen' : trainX_pos_cat.shape[2],  # pos classes (216)
    'capitallen' : trainX_capitals_cat.shape[2],  # capitalization classes (5)
    'ner_classes' : trainY_cat.shape[1],  # 18 
    'embed_dim' : global_embed_dim,  # word embedding size (300)
    'num_routing' : 3, 

    'use_glove' : True,
    'allow_glove_retrain' : False,
    'use_pos_tags' : True,
    'use_capitalization_info' : True,    
    
    'conv1_filters' : 256,
    'conv1_kernel_size' : 3,
    'conv1_strides' : 1,
    'conv1_padding' : 'valid',
    
    'conv2_filters' : 256,
    'conv2_kernel_size' : 3,
    'conv2_strides' : 1,
    'conv2_padding' : 'valid',
    
    'conv3_filters' : 128,
    'conv3_kernel_size' : 3,
    'conv3_strides' : 1,
    'conv3_padding' : 'valid',
    
    'max_pooling_size' : 3,
    'max_pooling_strides' : 1,
    'max_pooling_padding' : 'valid',
    'maxpool_dropout' : 0.3,
    
    'feed_forward_1' : 328,
    'ff1_dropout' : 0.3,
    'feed_forward_2' : 192,
    'ff2_dropout' : 0.3,
    
    'save_dir' : './result',
    'batch_size' : 100,
    'debug' : 2,
    'epochs' : 5,
    'stopping_patience' : 2, # default to same as epochs, ie don't use
    'dropout_p' : 0.25,
    'embed_dropout' : 0.25,  # set to 0 to disable dropout
    'lam_recon' : 0.0005,
    
    'optimizer' : 'Adam', #or 'SGD'
}

## Save All Data to Disk

In [28]:
# save all loaded data for use by training process
saveProcessedData( trainX, trainX_capitals_cat, trainX_pos_cat, devX, devX_capitals_cat,
                   devX_pos_cat, trainY_cat, devY_cat, embedding_matrix, train_decoderY, dev_decoderY)

## Model Training Functions

In [29]:
@timeit 
def trainModelSP( testFunc, modelName, hyper_params, embed_matrix=None, verbose=False):
    """
    testFunc - the name of the python file to run
    modelName - the internal name (ID) of the model to train
    hyper_params - a dict of hyper parameters
    """
    # save the hyperparams
    with open(HYPER_PARAM_FILE, mode='w') as fp:
        json.dump( hyper_params, fp)
    
    # call the train function
    # consider replacing with a call to subprocess!!
    !python {testFunc} {modelName} {HYPER_PARAM_FILE}


In [30]:
@timeit 
def testFeatures( testFunc, modelName, hyper_params):
    """
    builds and trains models for the configuration in hyper_params,
    1 for each input feature configuration: base, pos, caps, pos + caps 
      (no longer training pos and caps independently)
    
    testFunc - the name of the python file to run
    modelName - the model name to use for labeling
    """
    hypers = hyper_params.copy()
    
    # try the embeddings with different features
    
    # base
    curModel = modelName + "_" + LANGUAGE + "_" + TRAIN_AMOUNT + "_base"
    trainModelSP( testFunc, curModel, hypers )
    
    # pos tags
#     curModel = modelName + "_" + LANGUAGE + "_" + TRAIN_AMOUNT + "_features"
#     hypers['use_pos_tags'] = True
#     hypers['use_capitalization_info'] = False
#     trainModelSP( testFunc, curModel, hypers )
    
    # capitalization info
    #curModel = modelName + "_caps"
    #hypers['use_pos_tags'] = False
    #hypers['use_capitalization_info'] = True
    #trainModelSP( testFunc, curModel, hypers )
    
    # both
#     curModel = modelName + "_pos_caps"
#     hypers['use_pos_tags'] = True
#     hypers['use_capitalization_info'] = True
#     trainModelSP( testFunc, curModel, hypers )
    

##  Training
> the output isn't pretty, but we don't really need it since everything is stored in the history log. It is really just to show a sign of life.  
> * The below is just an example of how to set hyper parameters and train multiple models.

In [31]:
print("Preprocessing Time")
print("--- %s seconds ---" % (time.time() - start_time))

Preprocessing Time
--- 191.7661018371582 seconds ---


### train capsnet

In [32]:
# capsnet training function
testFunc = "trainCapsModel.py"

hypers = hyper_param_caps.copy()
hypers['epochs'] = 10
hypers['stopping_patience'] = 3
hypers['use_pos_tags'] = False
hypers['use_capitalization_info'] = False

# # try different embeddings
# # learn embeddings
# print("\n\nLearn Embeddings")
# hypers['use_glove'] = False
# hypers['embed_dropout'] = 0.0
# testFeatures( testFunc, "learn", hypers)

# # learn embeddings + Dropout
# print("\n\nLearn Embeddings and Dropout")
# hypers['use_glove'] = False
# hypers['embed_dropout'] = 0.25
# testFeatures( testFunc, "learn_dropout", hypers)

start_time_caps_1 = time.time()
# use glove, no learn
print("\n\nGlove Embeddings")
hypers['use_glove'] = True
hypers['allow_glove_retrain'] = False
hypers['embed_dropout'] = 0.0
testFeatures( testFunc, "glove_nolearn", hypers)

print("Training Time for caps glove no learn")
print("--- %s seconds ---" % (time.time() - start_time_caps_1))

# # use glove, no learn + Dropout
# print("\n\nGlove Embeddings and Dropout")
# hypers['use_glove'] = True
# hypers['allow_glove_retrain'] = False
# hypers['embed_dropout'] = 0.25
# testFeatures( testFunc, "glove_nolearn_dropout", hypers)

start_time_caps_2 = time.time()
# use glove, learn
print("\n\nGlove Embeddings with Learning")
hypers['use_glove'] = True
hypers['allow_glove_retrain'] = True
hypers['embed_dropout'] = 0.0
testFeatures( testFunc, "glove_learn", hypers)

print("Training Time for caps glove learn")
print("--- %s seconds ---" % (time.time() - start_time_caps_2))

# # # use glove, learn + Dropout
# print("\n\nGlove Embeddings with Learning and Dropout")
# hypers['use_glove'] = True
# hypers['allow_glove_retrain'] = True
# hypers['embed_dropout'] = 0.25
# testFeatures( testFunc, "glove_learn_dropout", hypers)




Glove Embeddings
Using TensorFlow backend.
fargs:
 ['trainCapsModel.py', 'glove_nolearn_es_10_base', 'hyper_params.json']
W0526 11:26:33.759031 139822604334912 module_wrapper.py:139] From /home/andrew/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0526 11:26:33.760782 139822604334912 module_wrapper.py:139] From /home/andrew/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0526 11:26:33.761124 139822604334912 module_wrapper.py:139] From /home/andrew/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0526 11:26:33.767339 139822604334912 module_wrapper.py:139] From /home/andrew/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.p

Epoch 2/10
40529/40529 [==============================] - 82s 2ms/step - loss: 0.0650 - acc: 0.9214 - val_loss: 0.0867 - val_acc: 0.8884

Epoch 00002: val_loss improved from 0.11243 to 0.08669, saving model to ./result/weights-02.h5
Epoch 3/10
40529/40529 [==============================] - 83s 2ms/step - loss: 0.0431 - acc: 0.9477 - val_loss: 0.0953 - val_acc: 0.8730

Epoch 00003: val_loss did not improve from 0.08669
Epoch 4/10
40529/40529 [==============================] - 81s 2ms/step - loss: 0.0334 - acc: 0.9592 - val_loss: 0.0843 - val_acc: 0.8872

Epoch 00004: val_loss improved from 0.08669 to 0.08429, saving model to ./result/weights-04.h5
Epoch 5/10
40529/40529 [==============================] - 81s 2ms/step - loss: 0.0278 - acc: 0.9667 - val_loss: 0.0879 - val_acc: 0.8865

Epoch 00005: val_loss did not improve from 0.08429
Epoch 6/10
40529/40529 [==============================] - 81s 2ms/step - loss: 0.0230 - acc: 0.9718 - val_loss: 0.0817 - val_acc: 0.8928

Epoch 00006: val_l

W0526 12:22:50.338221 140060443227968 deprecation.py:323] From /home/andrew/.local/lib/python3.7/site-packages/tensorflow_core/python/ops/math_grad.py:1424: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0526 12:22:50.832476 140060443227968 module_wrapper.py:139] From /home/andrew/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.

W0526 12:22:50.900979 140060443227968 module_wrapper.py:139] From /home/andrew/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:973: The name tf.assign is deprecated. Please use tf.compat.v1.assign instead.

Train on 40529 samples, validate on 52511 samples
W0526 12:22:52.570383 140060443227968 module_wrapper.py:139] From /home/andrew/.local/lib/python3.7/site-packages/keras/callbacks.p

### train CNN

In [33]:
# CNN training function
testFunc = "trainCNNModel.py"

hypers = hyper_param_cnn.copy()
hypers['epochs'] = 10
hypers['stopping_patience'] = 3
hypers['use_pos_tags'] = False
hypers['use_capitalization_info'] = False

# # try different embeddings
# # learn embeddings
# print("\n\nLearn Embeddings")
# hypers['use_glove'] = False
# hypers['embed_dropout'] = 0.0
# testFeatures( testFunc, "learn", hypers)

# # learn embeddings + Dropout
# print("\n\nLearn Embeddings and Dropout")
# hypers['use_glove'] = False
# hypers['embed_dropout'] = 0.25
# testFeatures( testFunc, "learn_dropout", hypers)

start_time_cnn_1 = time.time()
# # use glove, no learn
print("\n\nGlove Embeddings")
hypers['use_glove'] = True
hypers['allow_glove_retrain'] = False
hypers['embed_dropout'] = 0.0
testFeatures( testFunc, "glove_nolearn_cnn", hypers)

print("Training Time for cnn glove no learn")
print("--- %s seconds ---" % (time.time() - start_time_cnn_1))

# # use glove, no learn + Dropout
# print("\n\nGlove Embeddings and Dropout")
# hypers['use_glove'] = True
# hypers['allow_glove_retrain'] = False
# hypers['embed_dropout'] = 0.25
# testFeatures( testFunc, "glove_nolearn_dropout", hypers)

start_time_cnn_2 = time.time()
# use glove, learn
print("\n\nGlove Embeddings with Learning")
hypers['use_glove'] = True
hypers['allow_glove_retrain'] = True
hypers['embed_dropout'] = 0.0
testFeatures( testFunc, "glove_learn_cnn", hypers)

print("Training Time for cnn glove learn")
print("--- %s seconds ---" % (time.time() - start_time_cnn_2))

# # # use glove, learn + Dropout
# print("\n\nGlove Embeddings with Learning and Dropout")
# hypers['use_glove'] = True
# hypers['allow_glove_retrain'] = True
# hypers['embed_dropout'] = 0.25
# testFeatures( testFunc, "glove_learn_dropout", hypers)



Glove Embeddings
Using TensorFlow backend.
W0526 13:21:09.433722 140605240186688 module_wrapper.py:139] From /home/andrew/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0526 13:21:09.435643 140605240186688 module_wrapper.py:139] From /home/andrew/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0526 13:21:09.436007 140605240186688 module_wrapper.py:139] From /home/andrew/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0526 13:21:09.442503 140605240186688 module_wrapper.py:139] From /home/andrew/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.g

Epoch 1/10
40529/40529 [==============================] - 11s 274us/step - loss: 1.0762 - acc: 0.6535 - val_loss: 0.6223 - val_acc: 0.7961
W0526 13:29:28.111497 140605240186688 module_wrapper.py:139] From /home/andrew/.local/lib/python3.7/site-packages/keras/callbacks.py:995: The name tf.Summary is deprecated. Please use tf.compat.v1.Summary instead.


Epoch 00001: val_loss improved from inf to 0.62233, saving model to ./result/weights-01.h5
Epoch 2/10
40529/40529 [==============================] - 10s 252us/step - loss: 0.5575 - acc: 0.8178 - val_loss: 0.4713 - val_acc: 0.8484

Epoch 00002: val_loss improved from 0.62233 to 0.47126, saving model to ./result/weights-02.h5
Epoch 3/10
40529/40529 [==============================] - 11s 273us/step - loss: 0.4096 - acc: 0.8661 - val_loss: 0.4137 - val_acc: 0.8675

Epoch 00003: val_loss improved from 0.47126 to 0.41367, saving model to ./result/weights-03.h5
Epoch 4/10
40529/40529 [==============================] - 10s 253us/step - loss: 0.3

W0526 14:06:33.418215 140589891077952 deprecation.py:323] From /home/andrew/.local/lib/python3.7/site-packages/tensorflow_core/python/ops/math_grad.py:1424: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0526 14:06:33.568108 140589891077952 module_wrapper.py:139] From /home/andrew/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.

W0526 14:06:33.697057 140589891077952 module_wrapper.py:139] From /home/andrew/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:973: The name tf.assign is deprecated. Please use tf.compat.v1.assign instead.

Train on 40529 samples, validate on 52511 samples
W0526 14:06:35.686042 140589891077952 module_wrapper.py:139] From /home/andrew/.local/lib/python3.7/site-packages/keras/callbacks.p

In [34]:
# # SGD testing
# # capsnet training function
# testFunc = "trainCapsModel.py"

# hypers = hyper_param_caps.copy()
# hypers['optimizer'] = "SGD"
# print("Training with SGD - Nesterov Momentum Optimizer")

# hypers['epochs'] = 1
# hypers['stopping_patience'] = 3
# hypers['use_pos_tags'] = False
# hypers['use_capitalization_info'] = False

# # # try different embeddings
# # # learn embeddings
# # print("\n\nLearn Embeddings")
# # hypers['use_glove'] = False
# # hypers['embed_dropout'] = 0.0
# # testFeatures( testFunc, "SGD_primcaps_learn", hypers)

# # # learn embeddings + Dropout
# # print("\n\nLearn Embeddings and Dropout")
# # hypers['use_glove'] = False
# # hypers['embed_dropout'] = 0.25
# # testFeatures( testFunc, "SGD_primcaps_learn_dropout", hypers)

# # use glove, no learn
# print("\n\nGlove Embeddings")
# hypers['use_glove'] = True
# hypers['allow_glove_retrain'] = False
# hypers['embed_dropout'] = 0.0
# testFeatures( testFunc, "SGD_primcaps_glove_nolearn", hypers)

# # # use glove, no learn + Dropout
# # print("\n\nGlove Embeddings and Dropout")
# # hypers['use_glove'] = True
# # hypers['allow_glove_retrain'] = False
# # hypers['embed_dropout'] = 0.25
# # testFeatures( testFunc, "SGD_primcaps_glove_nolearn_dropout", hypers)

# # # use glove, learn
# # print("\n\nGlove Embeddings with Learning")
# # hypers['use_glove'] = True
# # hypers['allow_glove_retrain'] = True
# # hypers['embed_dropout'] = 0.0
# # testFeatures( testFunc, "SGD_primcaps_glove_learn", hypers)

# # # # use glove, learn + Dropout
# # print("\n\nGlove Embeddings with Learning and Dropout")
# # hypers['use_glove'] = True
# # hypers['allow_glove_retrain'] = True
# # hypers['embed_dropout'] = 0.25
# # testFeatures( testFunc, "SGD_primcaps_glove_learn_dropout", hypers)